In [3]:
pip install tensorflow==2.8.1 tensorflow-gpu==2.8.1 opencv-python mediapipe scikit-learn matplotlib

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [4]:
#เปลี่ยน BGR เป็น RGB
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [5]:
#กำหนดlandmark
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [6]:
#customตัวlandmark
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,244,121), thickness=2, circle_radius=2)
                             )
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [20]:
DATA_PATH= os.path.join('MP_Data')

#actions = np.array(['สวัสดี', 'รอ', 'กลับ', 'ขอบคุณ', 'ลง', 'ขึ้น', 'เฝ้า','คุย', 'ช่วย', 'เชื่อ', 'ฟัง', 'มอง', 'พา', 'ทักษิณ ชินวัตร', 'ขนม','แบก', 'อะไร', 'สมัคร', 'กระโดด', 'ยก', 'ชน', 'ผ่าน', 'แนะนำ', 'จำได้', 'ปลูก', 'รวม', 'ทำหาย', 'เจอ', 'หาย', 'พนักงานขาย', 'นักเขียนโปรแกรม', 'พ่อครัว', 'เจ้าหน้าที่ตำรวจ', 'เนื้อลูกแกะ', 'เนื้อหมู', 'คุณสบายดีไหม', 'ดูเหมือน', 'วาง', 'อยู่', 'ก', 'ข', 'ค', 'ฆ', 'ต', 'ถ', 'ง', 'จ', 'ฉ', 'ช', 'ซ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ท', 'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ'])
actions = np.array(['สวัสดี', 'รอ', 'กลับ', 'ขอบคุณ', 'ลง', 'ขึ้น', 'เฝ้า','คุย', 'ช่วย', 'เชื่อ', 'ฟัง', 'มอง', 'พา', 'ทักษิณ ชินวัตร', 'ขนม','แบก', 'อะไร', 'สมัคร', 'กระโดด', 'ยก', 'ชน', 'ผ่าน', 'แนะนำ', 'จำได้', 'ปลูก', 'รวม', 'ทำหาย', 'เจอ', 'หาย', 'พนักงานขาย', 'นักเขียนโปรแกรม', 'พ่อครัว', 'เจ้าหน้าที่ตำรวจ', 'เนื้อลูกแกะ', 'เนื้อหมู', 'คุณสบายดีไหม', 'ดูเหมือน', 'วาง', 'อยู่', 'ก', 'ข', 'ค', 'ง', 'จ', 'ช', 'อ'])

# 30 videos of data
no_sequences = 30

# videos are 30 frames length
sequence_length = 30

In [21]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [22]:
label_map = {label:num for num, label in enumerate(actions)}

In [23]:
label_map

{'สวัสดี': 0,
 'รอ': 1,
 'กลับ': 2,
 'ขอบคุณ': 3,
 'ลง': 4,
 'ขึ้น': 5,
 'เฝ้า': 6,
 'คุย': 7,
 'ช่วย': 8,
 'เชื่อ': 9,
 'ฟัง': 10,
 'มอง': 11,
 'พา': 12,
 'ทักษิณ ชินวัตร': 13,
 'ขนม': 14,
 'แบก': 15,
 'อะไร': 16,
 'สมัคร': 17,
 'กระโดด': 18,
 'ยก': 19,
 'ชน': 20,
 'ผ่าน': 21,
 'แนะนำ': 22,
 'จำได้': 23,
 'ปลูก': 24,
 'รวม': 25,
 'ทำหาย': 26,
 'เจอ': 27,
 'หาย': 28,
 'พนักงานขาย': 29,
 'นักเขียนโปรแกรม': 30,
 'พ่อครัว': 31,
 'เจ้าหน้าที่ตำรวจ': 32,
 'เนื้อลูกแกะ': 33,
 'เนื้อหมู': 34,
 'คุณสบายดีไหม': 35,
 'ดูเหมือน': 36,
 'วาง': 37,
 'อยู่': 38,
 'ก': 39,
 'ข': 40,
 'ค': 41,
 'ง': 42,
 'จ': 43,
 'ช': 44,
 'อ': 45}

In [24]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [69]:
len(sequences)

1380

In [70]:
X = np.array(sequences)

In [71]:
y = to_categorical(labels).astype(int)

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [73]:
y_train.shape

(69, 46)

In [80]:
y_train.shape

(1311, 46)

In [74]:
X_train.shape

(1311, 30, 1662)

In [79]:
X_test.shape

(69, 30, 1662)

In [66]:
X[0][0].shape

# Why is the format of this input 30 * 1662?

(1662,)

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [97]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

# model_alternative = Sequential()
# model_alternative.add(LSTM(256, return_sequences=False, activation='relu', input_shape=(30, 1662), dropout=0.2))
# model_alternative.add(Dense(3, activation='softmax')) #cheating on size
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [98]:
learning_rate = 0.00001

optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [99]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 30, 64)            442112    
_________________________________________________________________
lstm_22 (LSTM)               (None, 128)               98816     
_________________________________________________________________
dense_26 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_27 (Dense)             (None, 46)                2990      
Total params: 552,174
Trainable params: 552,174
Non-trainable params: 0
_________________________________________________________________


In [128]:
model.fit(X_train, y_train, epochs=25, callbacks=[tb_callback],validation_data=(X_test, y_test), batch_size=32)

Epoch 1/25
41/41 [==============================] - 5s 115ms/step - loss: 0.1243 - categorical_accuracy: 0.9611 - val_loss: 0.4501 - val_categorical_accuracy: 0.8841
Epoch 2/25
41/41 [==============================] - 3s 75ms/step - loss: 0.1423 - categorical_accuracy: 0.9588 - val_loss: 0.6551 - val_categorical_accuracy: 0.8551
Epoch 3/25
41/41 [==============================] - 3s 77ms/step - loss: 0.1248 - categorical_accuracy: 0.9626 - val_loss: 0.5316 - val_categorical_accuracy: 0.8406
Epoch 4/25
41/41 [==============================] - 3s 75ms/step - loss: 0.1173 - categorical_accuracy: 0.9672 - val_loss: 0.5916 - val_categorical_accuracy: 0.8551
Epoch 5/25
41/41 [==============================] - 3s 77ms/step - loss: 0.1125 - categorical_accuracy: 0.9703 - val_loss: 0.5477 - val_categorical_accuracy: 0.8406
Epoch 6/25
41/41 [==============================] - 4s 103ms/step - loss: 0.1360 - categorical_accuracy: 0.9611 - val_loss: 0.5718 - val_categorical_accuracy: 0.8551
Epoch 7/

KeyboardInterrupt: 

In [129]:
yhat = model.predict(X_test)

In [130]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [131]:
from sklearn.metrics import accuracy_score

In [132]:
accuracy_score(ytrue, yhat)

0.8840579710144928

In [133]:
from sklearn.metrics import ConfusionMatrixDisplay,multilabel_confusion_matrix, classification_report, confusion_matrix
# cm = multilabel_confusion_matrix(ytrue, yhat)
cm = confusion_matrix(ytrue, yhat)
cm

array([[3, 0, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 2]], dtype=int64)

In [134]:
print(classification_report(ytrue, yhat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         4
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         4
          16       1.00      1.00      1.00         2
          17       1.00      1.00      1.00         1
          18       1.00      1.00      1.00         3
          20       1.00      1.00      1.00         1
          21       1.00      1.00      1.00         1
          22       0.00    

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [123]:
plt.rcParams['figure.figsize'] = (80, 80)
plt.rcParams['figure.dpi'] = 100

fig, ax = plt.subplots()

disp = ConfusionMatrixDisplay(confusion_matrix=cm)

disp.plot(ax=ax)

plt.tight_layout()


plt.show()

In [135]:
model.save('action4.h5')


In [45]:
from tensorflow.keras.models import load_model

model = load_model('action2.h5')

# ใช้จริง

In [124]:
from PIL import ImageFont, Image, ImageDraw

In [125]:
from scipy import stats

In [126]:
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    return output_frame

In [23]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

TypeError: unsupported operand type(s) for -: 'method' and 'int'

<Figure size 1296x1296 with 0 Axes>

In [136]:
sequence = []
sentence = []
predictions = []
threshold = 0.95

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)

        fontpath = "./angsana.ttc"
        font = ImageFont.truetype(fontpath, 30)
        img_pil = Image.fromarray(image)
        draw = ImageDraw.Draw(img_pil)
        draw.text((30, 0),  ' '.join(sentence), font = font, fill = (255, 255, 255, 255))
        image = np.array(img_pil)
#         cv2.putText(image, ' '.join(sentence), (3,30), 
#                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
กลับ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
นักเขียนโปรแกรม
<class 'mediapipe.python.solution_base.SolutionOutputs'>
จ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
อยู่
<class 'mediapipe.python.solution_base.SolutionOutputs'>
อยู่
<class 'mediapipe.python.solution_base.SolutionOutputs'>
มอง
<class 'mediapipe.python.solution_base.SolutionOutputs'>
นักเขียนโปรแกรม
<class 'mediapipe.python.solution_base.SolutionOutputs'>
เจ้าหน้าที่ตำรวจ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
กลับ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
มอง
<class 'mediapipe.python.solution_base.SolutionOutputs'>
มอง
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สมัคร
<class 'mediapipe.python.solution_base.SolutionOutputs'>
สมัคร
<class 'mediapipe.python.solution_base.SolutionOutputs'>
มอง
<class 'mediapipe.python.solution_base.SolutionOutputs'>
เจ้าหน้าที่ตำรวจ
<class 'mediapipe.python.solu

In [19]:
print("Length of actions:", len(actions))
print("Index used:", np.argmax(res))
print("Length of sentence:", len(sentence))
print("Last element of sentence:", sentence[0])

Length of actions: 3
Index used: 2
Length of sentence: 0


IndexError: list index out of range